In [1]:
import numpy as np
import pandas as pd
import pickle
import re, nltk, spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from gensim.utils import simple_preprocess
from pprint import pprint
from sklearn.metrics.pairwise import euclidean_distances
from nltk.corpus import stopwords

In [2]:
dataset = pd.read_csv('../data/mortgage.csv')
dataset.head()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/28/19,Mortgage,Conventional home mortgage,Struggling to pay mortgage,None,"In dispute of the loan # XXXX, for XXXX XXXX t...",Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",GA,300XX,None,Consent provided,Web,03/28/19,Closed with explanation,Yes,NaN,3193709
1,02/21/19,Mortgage,FHA mortgage,Struggling to pay mortgage,None,My complaint is with Carrington Mortgage Servi...,None,"CARRINGTON MORTGAGE SERVICES, LLC",GA,312XX,None,Consent provided,Web,02/21/19,Closed with explanation,Yes,NaN,3158652
2,06/11/19,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,None,As of XX/XX/2019 PHH Mortgage Services purchas...,None,PHH Mortgage Services Corporation,FL,347XX,None,Consent provided,Web,06/11/19,Closed with explanation,Yes,NaN,3270600
3,11/29/19,Mortgage,Conventional home mortgage,Trouble during payment process,None,We started our mortgage with Loan Depot in XX/...,Company believes complaint is the result of an...,"LD Holdings Group, LLC",NJ,None,None,Consent provided,Web,11/29/19,Closed with monetary relief,Yes,NaN,3453669
4,10/23/19,Mortgage,Conventional home mortgage,Struggling to pay mortgage,None,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, XXXX ...",None,"Community Loan Servicing, LLC (formerly known ...",CA,92021,None,Consent provided,Web,11/05/19,Closed with explanation,Yes,NaN,3415392


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32805 entries, 0 to 32804
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 32805 non-null  object 
 1   Product                       32805 non-null  object 
 2   Sub-product                   32805 non-null  object 
 3   Issue                         32805 non-null  object 
 4   Sub-issue                     32805 non-null  object 
 5   Consumer complaint narrative  32805 non-null  object 
 6   Company public response       32805 non-null  object 
 7   Company                       32805 non-null  object 
 8   State                         32805 non-null  object 
 9   ZIP code                      32805 non-null  object 
 10  Tags                          32805 non-null  object 
 11  Consumer consent provided?    32805 non-null  object 
 12  Submitted via                 32805 non-null  object 
 13  D

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
dataset['Issue'].value_counts()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trouble during payment process                                                      15884
Struggling to pay mortgage                                                           7092
Applying for a mortgage or refinancing an existing mortgage                          5085
Closing on a mortgage                                                                3615
Incorrect information on your report                                                  677
Problem with a credit reporting company's investigation into an existing problem      335
Improper use of your report                                                            75
Unable to get your credit report or credit score                                       19
Credit monitoring or identity theft protection services                                16
Problem with fraud alerts or security freezes                                           7
Name: Issue, dtype: int64

In [5]:
dataset = dataset.drop(['Date received', 'Sub-issue', 'Company public response', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID'], axis=1)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Pre-processing

In [6]:
data = dataset['Consumer complaint narrative'].tolist()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Remove emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# lowecase
data = [sent.lower() for sent in data] #lowecase
# Remove newline 
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Tokenization

In [8]:
nltk.download("punkt")

tokenized_data = []
token_data = [nltk.word_tokenize(words) for words in data]
for words in token_data:
    new_words = [word for word in words if word.isalnum()]
    tokenized_data.append(new_words)
#ef sent_to_words(sentences):
#    for sentence in sentences:
#        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#tokenized_data = list(sent_to_words(data))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Stop words

In [9]:
#STOP WORDS
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'xxxx', 'xx', 'well', 'fargo', 'citibank', 'xxxxxx', 'xxxxxxxx'])
for key, value in dataset['Company'].value_counts().items():
    if value > 1000:
        word = key.lower().split(' ')
        for x in word:
            if len(x) > 1:
                stop_words.extend(x)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
tokenized_data = remove_stopwords(tokenized_data)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Pos tags

In [11]:
nltk.download('averaged_perceptron_tagger')
tagged_data = []
for token in tokenized_data:
    tagged_data.append(nltk.pos_tag(token))
#tagged = nltk.pos_tag(tokenized_data)
#singular_token = [word for word, tag in tagged if tag in ('NN', 'NNS', 'NNP', 'NNPS', 'PRP')]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
post_tag_words = []
for tagged in tagged_data:
    data_words = [word for word, tag in tagged if tag in ('NN', 'NNS', 'NNP', 'NNPS', 'PRP')]
    post_tag_words.append(data_words)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
post_tag_new = []
for word in post_tag_words:
    word = remove_values_from_list(word, 'xxxx')
    post_tag_new.append(word)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Lemmatization

In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(post_tag_new, allowed_postags=['NOUN','VERB']) #select noun and verb

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
data_lemm_token = [nltk.word_tokenize(word) for word in data_lemmatized]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Corpus

In [30]:
id2word = corpora.Dictionary(data_lemm_token)

# Create Corpus
texts = data_lemm_token

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 8), (1, 3), (2, 2), (3, 1), (4, 6), (5, 1), (6, 5), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 3), (17, 1), (18, 1), (19, 1), (20, 6), (21, 1), (22, 15), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1)]


## MODEL

In [32]:
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.073*"loan" + 0.062*"payment" + 0.028*"mortgage" + 0.021*"home" + '
  '0.014*"time" + 0.013*"month" + 0.012*"company" + 0.010*"letter" + '
  '0.010*"bank" + 0.009*"modification"'),
 (1,
  '0.048*"mortgage" + 0.030*"loan" + 0.024*"payment" + 0.021*"time" + '
  '0.016*"document" + 0.016*"home" + 0.015*"company" + 0.014*"letter" + '
  '0.014*"modification" + 0.014*"day"'),
 (2,
  '0.028*"payment" + 0.023*"home" + 0.021*"loan" + 0.020*"time" + '
  '0.017*"mortgage" + 0.016*"document" + 0.015*"property" + '
  '0.015*"information" + 0.015*"call" + 0.014*"company"'),
 (3,
  '0.050*"loan" + 0.049*"mortgage" + 0.027*"payment" + 0.022*"bank" + '
  '0.018*"time" + 0.016*"insurance" + 0.013*"day" + 0.012*"fee" + '
  '0.012*"amount" + 0.011*"call"'),
 (4,
  '0.040*"payment" + 0.032*"loan" + 0.020*"month" + 0.019*"account" + '
  '0.019*"bank" + 0.018*"check" + 0.017*"mortgage" + 0.016*"property" + '
  '0.016*"money" + 0.016*"insurance"'),
 (5,
  '0.072*"payment" + 0.039*"mortgage" + 0.036*"

In [33]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -6.202565031520746

Coherence Score:  nan
